# Imports and Data Read

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
from sklearn.neighbors import BallTree
from tqdm import tqdm
import plotly.express as px
import re
from haversine import haversine_vector, Unit

In [2]:
ct_data = pd.read_csv('CT_asset_cement_emissions.csv')
ets_data = pd.read_csv('EUETS_acquiring_accounts_bytransactions.csv')

cols = ['Acquiring.Holder.MainAddressLine', 'Acquiring.Holder.SecondaryAddressLine', 'Acquiring.Holder.City','Acquiring.Holder.ZipCode','Acquiring.Holder.Country']
ets_data['Address'] = ets_data[cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
ets_data['Address'] = ets_data['Address'].str.replace('nan, ','')
ct_data['st_astext'] = ct_data['st_astext'].astype(str)

ct_data[['lng', 'lat']] = ct_data['st_astext'].str.split(' ', 1, expand=True)
ct_data['lng'] = ct_data['lng'].str.replace('POINT(','',regex=False)
ct_data['lat'] = ct_data['lat'].str.replace(')','',regex=False)
ct_data['lat'] = ct_data['lat'].astype(float)
ct_data['lng'] = ct_data['lng'].astype(float)
codes = pd.read_excel('CountryCodes.xlsx')


/var/folders/h0/rfb20rk90qz0tdgv08kqcyrw0000gn/T/ipykernel_65426/1304844983.py:2: DtypeWarning: Columns (10,11,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  ets_data = pd.read_csv('EUETS_acquiring_accounts_bytransactions.csv')
/var/folders/h0/rfb20rk90qz0tdgv08kqcyrw0000gn/T/ipykernel_65426/1304844983.py:9: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  ct_data[['lng', 'lat']] = ct_data['st_astext'].str.split(' ', 1, expand=True)


# Filtering CT data for countries that are in ETS data

In [3]:
ct_data_2digcode = pd.merge(ct_data,codes,how='left',left_on='iso3_country',right_on='Code3')
ct_data_filterETS = ct_data_2digcode[ct_data_2digcode['Code2'].isin(ets_data['Acquiring.Holder.CountryCode'].unique())]
#ct_data_filterETS.to_csv('filteredETS_CTdata.csv')

# Geocoding all ETS addresses to Lat,Lng

In [29]:
locator = Nominatim(user_agent="myGeocoder")
unique_locs=ets_data['Address'].unique()
unique_locs

array(['Ziegeleistraße 14, Aschach an der Donau, 4082, Austria',
       'Bahnhofstrasse 10, Bregenz, 6900, Austria',
       'Weiberndorf 20, St. Johann i. Tirol, 6380, Austria', ...,
       'Bystrická cesta 1, Ružomberok, 3401, Slovakia',
       'Školská 470, Preseľany, 95612, Slovakia',
       'Pekná cesta 6, Bratislava, 83403, Slovakia'], dtype=object)

In [ ]:
latlngs = np.zeros((len(unique_locs),2))

for idx,loc in enumerate(tqdm(unique_locs)):
    try:
        gc = locator.geocode(loc)
        latlngs[idx,0]=gc.latitude
        latlngs[idx,1]=gc.longitude
    except:
        latlngs[idx,0]=91
        latlngs[idx,1]=91
#ets_data['point'] = ets_data['location'].apply(lambda loc: tuple(loc.point) if loc else None)
#ets_data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [ ]:
np.savetxt("GeocodedETSLatlngs.csv", latlngs, delimiter=",")

In [30]:
latlngs = np.loadtxt('GeocodedETSLatlngs.csv',delimiter=',')
#fig = px.scatter_geo(lat=latlngs[:,0], lon=latlngs[:,1],hover_name=unique_locs)

In [33]:
latlng_df = pd.DataFrame(data=latlngs,columns=['Lat','Lng'])
latlng_df['Address'] = unique_locs
latlng_df

,Lat,Lng,Address
0,48.372008,14.016811,"Ziegeleistraße 14, Aschach an der Donau, 4082,..."
1,47.503957,9.744931,"Bahnhofstrasse 10, Bregenz, 6900, Austria"
2,47.506653,12.401352,"Weiberndorf 20, St. Johann i. Tirol, 6380, Aus..."
3,47.815973,13.051648,"Louise-Piëch-Straße 2, Salzburg, 5020, Austria"
4,48.213625,16.414014,"Trabrennstraße 6-8, Wien, 1020, Austria"
...,...,...,...
13196,91.000000,91.000000,"Námestie A. Hlinka 3/816, Žilina, 1011, Slovakia"
13197,48.290522,18.540767,"Továrenská 210, Tlmače, 93528, Slovakia"
13198,49.076710,19.315823,"Bystrická cesta 1, Ružomberok, 3401, Slovakia"
13199,91.000000,91.000000,"Školská 470, Preseľany, 95612, Slovakia"


In [34]:
ets_data_geocoded = pd.merge(ets_data,latlng_df,how='left',on='Address')
ets_data_geocoded

,TransactionID,NbOfUnits,Acquiring.AccountIDRegistryCode,Acquiring.AccountID,Acquiring.RegistryCode,Acquiring.NationalAdministrator,Acquiring.AccountStatus,Acquiring.AccountOpeningDate,Acquiring.AccountType,Acquiring.RelatedInstallationAircraftOperatorID,...,Acquiring.Holder.City,Acquiring.Holder.SecondaryAddressLine,Acquiring.Holder.RelationshipType,Acquiring.Holder.CountryCode,Acquiring.Holder.Country,Acquiring.Holder.ZipCode,Acquiring.Holder.MainAddressLine,Address,Lat,Lng
0,FR21168,2000,AT10621,10621.0,AT,Austria,closed,2005-12-29 00:00:00.0,Former Operator Holding Account,NaN,...,Aschach an der Donau,Ziegeleistraße 14,Account holder,AT,Austria,4082,NaN,"Ziegeleistraße 14, Aschach an der Donau, 4082,...",48.372008,14.016811
1,AT8881,13646,AT10621,10621.0,AT,Austria,closed,2005-12-29 00:00:00.0,Former Operator Holding Account,NaN,...,Aschach an der Donau,Ziegeleistraße 14,Account holder,AT,Austria,4082,NaN,"Ziegeleistraße 14, Aschach an der Donau, 4082,...",48.372008,14.016811
2,AT13722,13646,AT10621,10621.0,AT,Austria,closed,2005-12-29 00:00:00.0,Former Operator Holding Account,NaN,...,Aschach an der Donau,Ziegeleistraße 14,Account holder,AT,Austria,4082,NaN,"Ziegeleistraße 14, Aschach an der Donau, 4082,...",48.372008,14.016811
3,AT7617,13646,AT10621,10621.0,AT,Austria,closed,2005-12-29 00:00:00.0,Former Operator Holding Account,NaN,...,Aschach an der Donau,Ziegeleistraße 14,Account holder,AT,Austria,4082,NaN,"Ziegeleistraße 14, Aschach an der Donau, 4082,...",48.372008,14.016811
4,AT17534,13646,AT10621,10621.0,AT,Austria,closed,2005-12-29 00:00:00.0,Former Operator Holding Account,NaN,...,Aschach an der Donau,Ziegeleistraße 14,Account holder,AT,Austria,4082,NaN,"Ziegeleistraße 14, Aschach an der Donau, 4082,...",48.372008,14.016811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046617,EU217344,41645,XINA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,46.314475,11.048029
1046618,EU46951,53467,XINA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,46.314475,11.048029
1046619,EU131484,51843,XINA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,46.314475,11.048029
1046620,EU50451,32000,XINA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,46.314475,11.048029


In [45]:
ets_data_geocoded.loc[(ets_data_geocoded['Address'] == 'nan'),['Lat','Lng']] = np.nan
ets_data_geocoded.loc[(ets_data_geocoded['Lat'] == 91),['Lat','Lng']] = np.nan
#Around 10% have NAN addresses

0    48.372008
1    48.372008
2    48.372008
3    48.372008
4    48.372008
Name: Lat, dtype: float64

## TODO:
* Reverse geocode all CT points
* Match CT and ETS on country AND zip code
* Generate distances between all matches pairs
* For distances below 5km

# Reverse Geocode all CT points to get Country,Zip

In [7]:
ct_addresses = np.zeros(len(ct_data.lat),dtype=str)

for idx,loc in enumerate(tqdm(ct_data.lat)):
    try:
        ct_addresses[idx]=(locator.reverse(str(ct_data.lat[idx])+','+str(ct_data.lng[idx])))
    except:
        ct_addresses[idx]='NA'
    print(ct_addresses)
    break
#ets_data['point'] = ets_data['location'].apply(lambda loc: tuple(loc.point) if loc else None)
#ets_data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

  0%|                                                                                                                                             | 0/992200 [00:00<?, ?it/s]

['N' '' '' ... '' '' '']


# Generate distance between all pairs of CT lat,lng and ETS lat,lng in same country

## TODO:
* Split CT and ETS into country blocks
* For each matching country block, calculate haversine

In [48]:
ets_country_blocks = []
ct_country_blocks = []

all_countries_ct = ct_data_filterETS['Code2'].unique()
all_countries_ets = ets_data_geocoded['Acquiring.Holder.CountryCode'].unique()

for country in all_countries_ct:
    ct_country_blocks.append(ct_data_filterETS[ct_data_filterETS['Code2']==country])
    ets_country_blocks.append(ets_data_geocoded[ets_data_geocoded['Acquiring.Holder.CountryCode']==country])

In [54]:
ets_country_blocks[63][['Lat','Lng']].unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [51]:
ct_country_blocks[63]

,asset_id,iso3_country,original_inventory_sector,start_time,end_time,temporal_granularity,gas,emissions_quantity,emissions_factor,emissions_factor_units,...,created_date,modified_date,asset_name,asset_type,st_astext,lng,lat,Country,Code2,Code3
252095,1754104,AUT,cement,2016-08-01 00:00:00,2016-08-31 00:00:00,month,ch4,0,NaN,tonnes_gas_per_tonnesCement,...,2022-09-05 15:22:06.535075,NaN,Wopfing cement plant,Dry,POINT(16.085765 47.873176),16.085765,47.873176,Austria,AT,AUT
252096,1754104,AUT,cement,2016-09-01 00:00:00,2016-09-30 00:00:00,month,ch4,0,NaN,tonnes_gas_per_tonnesCement,...,2022-09-05 15:22:06.535075,NaN,Wopfing cement plant,Dry,POINT(16.085765 47.873176),16.085765,47.873176,Austria,AT,AUT
252097,1754104,AUT,cement,2016-10-01 00:00:00,2016-10-31 00:00:00,month,ch4,0,NaN,tonnes_gas_per_tonnesCement,...,2022-09-05 15:22:06.535075,NaN,Wopfing cement plant,Dry,POINT(16.085765 47.873176),16.085765,47.873176,Austria,AT,AUT
252098,1754104,AUT,cement,2016-11-01 00:00:00,2016-11-30 00:00:00,month,ch4,0,NaN,tonnes_gas_per_tonnesCement,...,2022-09-05 15:22:06.535075,NaN,Wopfing cement plant,Dry,POINT(16.085765 47.873176),16.085765,47.873176,Austria,AT,AUT
252099,1754104,AUT,cement,2016-12-01 00:00:00,2016-12-31 00:00:00,month,ch4,0,NaN,tonnes_gas_per_tonnesCement,...,2022-09-05 15:22:06.535075,NaN,Wopfing cement plant,Dry,POINT(16.085765 47.873176),16.085765,47.873176,Austria,AT,AUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892254,1754107,AUT,cement,2015-02-01 00:00:00,2015-02-28 00:00:00,month,co2,20895,0.777538,tonnes_gas_per_tonnesCement,...,2022-09-05 15:22:06.535932,NaN,Retznei cement plant,Dry,POINT(15.573051 46.737096),15.573051,46.737096,Austria,AT,AUT
892255,1754107,AUT,cement,2015-02-01 00:00:00,2015-02-28 00:00:00,month,n2o,0,NaN,tonnes_gas_per_tonnesCement,...,2022-09-05 15:22:06.535932,NaN,Retznei cement plant,Dry,POINT(15.573051 46.737096),15.573051,46.737096,Austria,AT,AUT
892256,1754107,AUT,cement,2015-02-01 00:00:00,2015-02-28 00:00:00,month,co2e_100yr,20895,NaN,tonnes_gas_per_tonnesCement,...,2022-09-05 15:22:06.535932,NaN,Retznei cement plant,Dry,POINT(15.573051 46.737096),15.573051,46.737096,Austria,AT,AUT
892257,1754107,AUT,cement,2015-02-01 00:00:00,2015-02-28 00:00:00,month,co2e_20yr,20895,NaN,tonnes_gas_per_tonnesCement,...,2022-09-05 15:22:06.535932,NaN,Retznei cement plant,Dry,POINT(15.573051 46.737096),15.573051,46.737096,Austria,AT,AUT


In [34]:
haversine_vector(ETS_countryblock, CT_countryblock, Unit.KILOMETERS, comb=True)
len(ct_data_filterETS)*len(latlngs)

923560

13201